In [8]:
## IMPORTANDO LIBS
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
#!pip install tensorflow
#!pip install tflearn
import tflearn
import tensorflow as tf
import random
import json
import string
import unicodedata
import sys
import pandas as pd
from sklearn.metrics import accuracy_score
import random
from random import shuffle


In [3]:
### CRIANDO FUNÇÃO PARA REMOVER PONTUAÇÕES

# criando tabela com códigos das pontuações
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                    if unicodedata.category(chr(i)).startswith('P'))

# função para remover pontuações
def remove_punctuation(text):
    return text.translate(tbl)

# inicializando objeto stemmer
stemmer = LancasterStemmer()

In [4]:
## CARREGANDO OS DADOS
dados = pd.read_csv('~/Documents/dsa/data/reviewFinal.csv',encoding='latin-1')
corpus = pd.DataFrame()
listTitle = []
listRecom = []

## FILTRANDO OS REGISTROS
for i, r in dados.iterrows():
    if r['recommend'] != 'NI' and r['title'] != "" and r['title'] is not np.nan and r['site'] == "glassdoor":
        listTitle.append(r['title'])
        listRecom.append(r['recommend'])


In [9]:
corpus['title'] = listTitle
corpus['recommend'] = listRecom

## CRIANDO LINHAS PARA SELECIONAR BASE DE TESTE E BASE DE TREINO
x = [ i for i in range(len(corpus))]
shuffle(x)

prc_train  = 0.75 ## variavel para indicar a porcentagem para a base de treino
rows_train = x[0:round( len(corpus) * prc_train)]
rows_test  = x[round( len(corpus) * prc_train):(len(corpus))]

data = corpus.iloc[rows_train, ]

In [10]:
# LISTANDO TODOS OS VALORES PARA ATRIBUTO 'RECOMMEND'
categories = list(set(listRecom))
words = []
docs  = []


#loop para percorrer o DF, tratar os dados e salvar no objetos WORDS e DOCS
for i, row in data.iterrows():
    
    each_sentence = remove_punctuation(row['title'])

    w = nltk.word_tokenize(each_sentence)
    words.extend(w)
    
    docs.append((w, row['recommend']))


In [13]:
# CONVERTENDO TERMOS PARA STEMMER E MINUSCULO E REMOVENDO DUPLICIDADE
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words]
words = sorted(list(set(words)))

In [14]:
#CRIANDO BASE DE TREINO 
training = []
output   = []

# create an empty array for our output
output_empty = [0] * len(categories)


In [15]:
for doc in docs:
    # inicializando bag of words(bow) para cada documento
    bow = []
    # list de tokenized words
    token_words = doc[0]
    # stem de cada palavra
    token_words = [stemmer.stem(word.lower()) for word in token_words]
    # adicionando termo na bag of word(bow)
    for w in words:
        bow.append(1) if w in token_words else bow.append(0)

    output_row = list(output_empty)
    output_row[categories.index(doc[1])] = 1

    #adicionando a lista de treinando os termos com sua respectiva classe
    training.append([bow, output_row])


In [16]:
# misturando os valores na base de treino e convertendo em um array
random.shuffle(training)
training = np.array(training)

#criando lista de treino para valores X(termos) e Y(classe)
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [17]:
# reset underlying graph data
tf.reset_default_graph()

# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# definindo o modelo e as configuraçoes
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# treinando o modelo (aplicando gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=5, batch_size=8, show_metric=True)
#model.save('model.tflearn')

Training Step: 40289  | total loss: 0.44550 | time: 38.944s
| Adam | epoch: 005 | loss: 0.44550 - acc: 0.7846 -- iter: 64456/64460
Training Step: 40290  | total loss: 0.44429 | time: 38.948s
| Adam | epoch: 005 | loss: 0.44429 - acc: 0.7811 -- iter: 64460/64460
--


In [18]:
#FUNÇÃO PARA FAZER TRATAMENTO DOS TERMOS QUE SERÃO CLASSIFICADOS
def get_tf_record(sentenceL):    
    global words
    retorno = []
    if type(sentenceL) == list :
        for sentence in sentenceL:
            sentence_words = nltk.word_tokenize(sentence)
            sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
            bow = [0]*len(words)
            for s in sentence_words:
                for i, w in enumerate(words):
                    if w == s:
                        bow[i] = 1
            retorno.append(bow)
        return(np.array(retorno))
    else:
        sentence = sentenceL        
        sentence_words = nltk.word_tokenize(sentence)        
        sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
        
        bow = [0]*len(words)
        for s in sentence_words:
            for i, w in enumerate(words):
                if w == s:
                    bow[i] = 1
        retorno.append(bow)

        return(np.array(bow))
        

In [20]:
#CRIANDO BASE DE TESTE
data_test  = corpus.iloc[rows_test,]
test_x = []
test_y = []

for i,x in data_test.iterrows():
    test_x.append(x['title'])
    test_y.append(categories.index(x['recommend']))
    
test_x_clean = get_tf_record(test_x)

pred   = model.predict(test_x_clean)
pred_y = list(np.argmax(pred,axis=1))

## VERFICIANDO ACURACIA DO MODELO
accuracy_score(test_y,pred_y)

0.7661267802289863